# Building Network

## Step 2: Building a Model
### Instantiating a small covnet for controls vs patients 

In [1]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(90, 80, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
    

Using TensorFlow backend.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 78, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 44, 39, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 8, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 6, 128)         147584    
__________

### Configuring model for training

In [3]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-4),
metrics=['acc'])

## Step 3: Data preprocessing 

### - Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_dir= '/Code/dua/602020_Split/small sample/Relocated_Data2/Train'
validation_dir= '/Code/dua/602020_Split/small sample/Relocated_Data2/Validation'

#Rescales all images by 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
         train_dir,                 #Target directory
         target_size = (90, 80),  #Resizes all images to 150 × 150
         batch_size = 100,
         class_mode = 'binary')     #Because you use binary_crossentropy loss, you need binary labels.
validation_generator = test_datagen.flow_from_directory(
         validation_dir,          #Target directory
         target_size =(90, 80),  #Resizes all images to 150 × 150
         batch_size = 100,           #Because you use binary_crossentropy loss, you need binary labels.  
         class_mode = 'binary')

# Break loop at some point to stop from having batches run indefinitely
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
data batch shape: (100, 90, 80, 3)
labels batch shape: (100,)


In [5]:
history = model.fit_generator(
train_generator,
steps_per_epoch=1632,
epochs=30,
validation_data=validation_generator,
validation_steps=340)

Epoch 1/30
1632/1632 [==============================] - 796s 488ms/step - loss: 0.0761 - acc: 0.9685 - val_loss: 6.3517 - val_acc: 0.5048
Epoch 2/30
1632/1632 [==============================] - 794s 487ms/step - loss: 5.8428e-04 - acc: 0.9998 - val_loss: 7.2889 - val_acc: 0.5047
Epoch 3/30
1632/1632 [==============================] - 794s 487ms/step - loss: 1.3594e-07 - acc: 1.0000 - val_loss: 7.5990 - val_acc: 0.5050
Epoch 4/30
1632/1632 [==============================] - 794s 487ms/step - loss: 1.0962e-07 - acc: 1.0000 - val_loss: 7.6085 - val_acc: 0.5051
Epoch 5/30
1632/1632 [==============================] - 797s 488ms/step - loss: 1.0961e-07 - acc: 1.0000 - val_loss: 7.6090 - val_acc: 0.5051
Epoch 6/30
1632/1632 [==============================] - 795s 487ms/step - loss: 1.0960e-07 - acc: 1.0000 - val_loss: 7.6153 - val_acc: 0.5047
Epoch 7/30
1632/1632 [==============================] - 798s 489ms/step - loss: 1.0961e-07 - acc: 1.0000 - val_loss: 7.6124 - val_acc: 0.5049
Epoch 8/30

In [6]:
model.save('Controls_and_Patients_small.h5')
model.save_weights('Controls_and_Patients_small_Weights.h5')

In [1]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined